# Top Model Configurations

**Import Packages**  
The following cell will import the required packages, and print the their current version, and indicate how many GPU's are connected to the system.

In [4]:
import os
import time
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import Xception, InceptionV3, ResNet50, VGG16, VGG19
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Pandas version:", pd.__version__)
print("Tensorflow version:", tf.__version__)
print("Number GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Tensorflow version: 2.12.0
Num GPUs Available:  1


In [2]:
def create_train_val_datasets(
    preprocessing_model,
    input_dir,
    output_dir="data/datasets",
    img_height=299,
    img_width=299,
    batch_size=32,
    save_dataset=False,
    augment_training_data=False
):
    """
    Creates train and validation datasets from images in the input directory.

    Args:
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories.
        output_dir (str): The directory where the TFRecord files will be saved. Default is 'data/datasets'.
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.
        save_dataset (bool): Whether to save the datasets as TFRecord files. Default is True.

    Returns:
        tf.data.Dataset, tf.data.Dataset: The train and validation datasets.
    """
    # Create ImageDataGenerator with validation split
    if augment_training_data:
        datagenTrain = ImageDataGenerator(
                    preprocessing_function=preprocessing_model,
                    rotation_range=20,  # Increased rotation range
                    width_shift_range=0.2,  # Increased shift range
                    height_shift_range=0.2,  # Increased shift range
                    shear_range=0.2,  # Increased shear range
                    zoom_range=0.2,  # Increased zoom range
                    horizontal_flip=True,
                    fill_mode="nearest",
                    validation_split=0.2
        )
    else:
        datagenTrain = ImageDataGenerator(
                    preprocessing_function=preprocessing_model,
                    validation_split=0.2
        )
        
    datagenVal = ImageDataGenerator(
                preprocessing_function=preprocessing_model,
                validation_split=0.2
    )   

        
    
    # Create a train and validation data generator
    train_gen = datagenTrain.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="training",
        shuffle=True
    )

    val_gen = datagenVal.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="validation",
        shuffle=True
    )
    
    # Save the dataset if requested
    if save_dataset:
        train_filename = generate_filename("train_ds")
        val_filename = generate_filename("val_ds")
        tf.data.Dataset.save(train_gen, f"{output_dir}/{train_filename}.tfrecords")
        tf.data.Dataset.save(val_gen, f"{output_dir}/{val_filename}.tfrecords")
    
    return train_gen, val_gen

def create_test_dataset(
    preprocessing_model,
    input_dir,
    output_dir="data/datasets",
    img_height=299,
    img_width=299,
    batch_size=32,
    save_dataset=False
):
    """
    Creates a test dataset from images in the input directory.

    Args:
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories.
        output_dir (str): The directory where the TFRecord file will be saved. Default is 'data/datasets'.
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.
        save_dataset (bool): Whether to save the dataset as a TFRecord file. Default is True.

    Returns:
        tf.data.Dataset: The test dataset.
    """
    # Create ImageDataGenerator
    datagen = ImageDataGenerator(
        preprocessing_function=preprocessing_model
    )

    # Load all images
    test_gen = datagen.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True
    )
    
    if save_dataset:
        filename = generate_filename("test_ds")
        tf.data.Dataset.save(test_gen, f"{output_dir}/{filename}.tfrecords")

    return test_gen

# Experiments
---
## Experiment #0001 | Initial Model Selection
**Experiment ID:** 0001  
**Experiment Description:** Initial testing to select a model, code cycles through various pre-trained CNN models and records there binary accuracy and loss on the training, validation and testing datasets. The classification head contains a single layer. Testing is also conducted with layers frozen and un-frozen.
  
**Experiment Outcome:** VGG model performed poorly, when layers were unfrozen, all models indicated over-fitting of the data.
**Next Test:** The next test to be performed is ID0002, We propose adding an extra dropout layer and dropping the VGG models. 


In [ ]:
# Experiment ID:0001
experimentId = "0001"

# Load Directories
train_directory = "/home/luke/datasets/train-5"
test_directory = "/home/luke/datasets/test-5"
experiment_directory = f"/home/luke/Development/deepfake-detection/experiments/model-selection/{experimentId}"
results_directory = f"{experiment_directory}/results"

# Create directories if they don't exist
os.makedirs(experiment_directory, exist_ok=True)
os.makedirs(results_directory, exist_ok=True)

# Declare accuracy_test list
accuracy_test = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=3
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
)

# Loop through all combinations of models and trainable layers
for pretrained_model, preprocessing_model, isTrainable in [(Xception, xception, True),
                                                        (Xception, xception, False),
                                                        (InceptionV3, inception_v3, True),
                                                        (InceptionV3, inception_v3, False),
                                                        (ResNet50, resnet50, True),
                                                        (ResNet50, resnet50, False),
                                                        (VGG16, vgg16, True),
                                                        (VGG16, vgg16, False),
                                                        (VGG19, vgg19, True),
                                                        (VGG19, vgg19, False)]:
    
    # Setup Model Checkpoint Callback
    path_trainable = "trainable" if isTrainable else "untrainable"
    model_checkpoint = ModelCheckpoint(
    f'{experiment_directory}/models/{pretrained_model.__name__}/{path_trainable}/best_model.h5', 
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    save_best_only=True
    )

    # Start Timer to measure processing time 
    start_time = time.time()

    # Create a dataset and preprocess images to suit base model
    train_ds, val_ds = create_train_val_datasets(preprocessing_model.preprocess_input, train_directory, batch_size=16)
    test_ds = create_test_dataset(preprocessing_model.preprocess_input, test_directory, batch_size=16)

    # Create the base model from the pre-trained model
    base_model = pretrained_model(
        weights="imagenet", 
        include_top=False,
        input_tensor=Input(shape=(299, 299, 3)))

    # Freeze/Unfreeze the base model layers
    base_model.trainable = isTrainable

    # Add a classification head
    top_model = base_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    predictions = Dense(1, activation='sigmoid')(top_model)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(
        optimizer=Adam(1e-3),  
        loss=BinaryCrossentropy(from_logits=False),
        metrics=[BinaryAccuracy()],
    )

    # Train the model
    history = model.fit(
        train_ds, 
        steps_per_epoch=train_ds.samples // train_ds.batch_size,
        epochs=50, 
        validation_data=val_ds,
        validation_steps=val_ds.samples // val_ds.batch_size,
        callbacks = [early_stopping, lr_reducer, model_checkpoint]
    )
    
    # End Timer and calculate processing time
    end_time = time.time()
    processing_time = end_time - start_time

    # Evaluate the model
    evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

    # Append results to accuracy_test list
    accuracy_test.append({
        'base_model': pretrained_model.__name__,
        'base_layers_trainable': isTrainable,   
        'processing_time': round(processing_time, 3),
        'test_accuracy': round(evaluate[1], 3),
        'test_loss': round(evaluate[0], 3),
        'train_accuracy': round(history.history['binary_accuracy'][-1],3),
        'train_loss': round(history.history['loss'][-1],3),
        'val_accuracy': round(history.history['val_binary_accuracy'][-1],3),
        'val_loss': round(history.history['val_loss'][-1],3)
    })

# Create DataFrame with accuracy score
accuracy_test_results = pd.DataFrame(accuracy_test)
accuracy_test_results.to_csv('{experiment_directory}/results/accuracy_test_results.csv', index=False)

----
## Experiment #0002 | Initial Model Selection
**Experiment ID:** 0002  
**Experiment Description:** Iteration 2 to select a model, code cycles through various pre-trained CNN models and records there binary accuracy and loss on the training, validation and testing datasets. The classification head contains two layers. Testing is also conducted with layers frozen and un-frozen. Note that from test 0001 the VGG model has been removed.
  
**Experiment Outcome:** Tests performed better overall, over-fittings seems to have reduced, happy to commence with Xception
**Next Test:** Next test will be ID#0003, I will continue with the same configuration except start at 5 unfrozen layers and work my way up until the entire model is unfrozen recording accuracy along the way.

In [5]:
# Experiment ID:0002
# Experiment Description: Same as 0001 but due to poor performance of VGG models, they were dropped, also added a dropout layer to help prevent over-fitting.
experimentId = "0002"

# Load Directories
train_directory = "/home/luke/datasets/train-5"
test_directory = "/home/luke/datasets/test-5"
experiment_directory = f"/home/luke/Development/deepfake-detection/experiments/model-selection/{experimentId}"
results_directory = f"{experiment_directory}/results"

# Create directories if they don't exist
os.makedirs(experiment_directory, exist_ok=True)
os.makedirs(results_directory, exist_ok=True)

# Declare accuracy_test list
accuracy_test = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=3
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
)

# Loop through all combinations of models and trainable layers
for pretrained_model, preprocessing_model, isTrainable in [(Xception, xception, True),
                                                        (Xception, xception, False),
                                                        (InceptionV3, inception_v3, True),
                                                        (InceptionV3, inception_v3, False),
                                                        (ResNet50, resnet50, True),
                                                        (ResNet50, resnet50, False),
                                                        # (VGG16, vgg16, True),
                                                        # (VGG16, vgg16, False),
                                                        # (VGG19, vgg19, True),
                                                        # (VGG19, vgg19, False)
                                                        ]:
    
    # Setup Model Checkpoint Callback
    path_trainable = "trainable" if isTrainable else "untrainable"
    model_checkpoint = ModelCheckpoint(
    f'{experiment_directory}/models/{pretrained_model.__name__}/{path_trainable}/best_model.h5', 
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    save_best_only=True
    )

    # Start Timer to measure processing time 
    start_time = time.time()

    # Create a dataset and preprocess images to suit base model
    train_ds, val_ds = create_train_val_datasets(preprocessing_model.preprocess_input, train_directory, batch_size=16)
    test_ds = create_test_dataset(preprocessing_model.preprocess_input, test_directory, batch_size=16)

    # Create the base model from the pre-trained model
    base_model = pretrained_model(
        weights="imagenet", 
        include_top=False,
        input_tensor=Input(shape=(299, 299, 3)))

    # Freeze/Unfreeze the base model layers
    base_model.trainable = isTrainable

    # Add a classification head
    top_model = base_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dropout(0.5)(top_model)
    predictions = Dense(1, activation='sigmoid')(top_model)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(
        optimizer=Adam(1e-3),  
        loss=BinaryCrossentropy(from_logits=False),
        metrics=[BinaryAccuracy()],
    )

    # Train the model
    history = model.fit(
        train_ds, 
        steps_per_epoch=train_ds.samples // train_ds.batch_size,
        epochs=50, 
        validation_data=val_ds,
        validation_steps=val_ds.samples // val_ds.batch_size,
        callbacks = [early_stopping, lr_reducer, model_checkpoint]
    )
    
    # End Timer and calculate processing time
    end_time = time.time()
    processing_time = end_time - start_time

    # Evaluate the model
    evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

    # Append results to accuracy_test list
    accuracy_test.append({
        'base_model': pretrained_model.__name__,
        'base_layers_trainable': isTrainable,   
        'processing_time': round(processing_time, 3),
        'test_accuracy': round(evaluate[1], 3),
        'test_loss': round(evaluate[0], 3),
        'train_accuracy': round(history.history['binary_accuracy'][-1],3),
        'train_loss': round(history.history['loss'][-1],3),
        'val_accuracy': round(history.history['val_binary_accuracy'][-1],3),
        'val_loss': round(history.history['val_loss'][-1],3)
    })

# Create DataFrame with accuracy score
accuracy_test_results = pd.DataFrame(accuracy_test)
accuracy_test_results.to_csv(f'{results_directory}/accuracy_test_results.csv', index=False)

Found 9660 images belonging to 2 classes.
Found 2414 images belonging to 2 classes.
Found 2139 images belonging to 2 classes.


2023-05-24 12:40:15.151261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-24 12:40:15.151599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-24 12:40:15.151892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/50


2023-05-24 12:40:16.324641: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-24 12:40:21.051902: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-24 12:40:22.554740: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-24 12:40:22.555315: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x55b6fddffca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-24 12:40:22.555328: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2023-05-24 12:40:22.55868

603/603 [==============================] - ETA: 0s - loss: 0.3849 - binary_accuracy: 0.8268

...

602/603 [============================>.] - ETA: 0s - loss: 0.7174 - binary_accuracy: 0.5675

2023-05-24 13:20:21.286191: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 0.66373, saving model to /home/luke/Development/deepfake-detection/experiments/0002/models/ResNet50/untrainable/best_model.h5
603/603 [==============================] - 37s 59ms/step - loss: 0.7176 - binary_accuracy: 0.5674 - val_loss: 0.6637 - val_binary_accuracy: 0.5754 - lr: 0.0010
Epoch 2/50
603/603 [==============================] - ETA: 0s - loss: 0.6571 - binary_accuracy: 0.6189
Epoch 2: val_loss improved from 0.66373 to 0.65550, saving model to /home/luke/Development/deepfake-detection/experiments/0002/models/ResNet50/untrainable/best_model.h5
603/603 [==============================] - 35s 58ms/step - loss: 0.6571 - binary_accuracy: 0.6189 - val_loss: 0.6555 - val_binary_accuracy: 0.5992 - lr: 0.0010
Epoch 3/50
602/603 [============================>.] - ETA: 0s - loss: 0.6331 - binary_accuracy: 0.6408
Epoch 3: val_loss did not improve from 0.65550
603/603 [==============================] - 35s 59ms/step - loss: 0.6332 - binary_accuracy: 0

2023-05-24 13:25:08.519428: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


133/133 [==============================] - 6s 47ms/step - loss: 0.6150 - binary_accuracy: 0.6588


----
## Experiment #0003 | Initial Model Selection
**Experiment ID:** 0003  
**Experiment Description:** Extension of test #0002, except we have eliminated resnet, now we will cycle through the two models unfreezing 5 layers at a time, attempting to find the best fit. Tests are capped at 4 epochs.
  
**Experiment Outcome:**   
**Next Test:**  No further testing required for model selection.

In [7]:
# Experiment ID:0003
experimentId = "0003"

# Load Directories
train_directory = "/home/luke/datasets/train-5"
test_directory = "/home/luke/datasets/test-5"
experiment_directory = f"/home/luke/Development/deepfake-detection/experiments/model-selection/{experimentId}"
results_directory = f"{experiment_directory}/results"

# Create directories if they don't exist
os.makedirs(experiment_directory, exist_ok=True)
os.makedirs(results_directory, exist_ok=True)

# Declare accuracy_test list
accuracy_test = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=3
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1,
)


# Loop through all combinations of models and trainable layers
for pretrained_model, preprocessing_model in [(Xception, xception),
                                              (InceptionV3, inception_v3)]:

    # Create the base model from the pre-trained model
    base_model = pretrained_model(
        weights="imagenet", 
        include_top=False,
        input_tensor=Input(shape=(299, 299, 3)))

    # Initial freezing of all layers
    for layer in base_model.layers:
        layer.trainable = False

    # Determine how many sets of 5 layers exist in the model
    n_sets = len(base_model.layers) // 5

    for i in range(n_sets):
        # Unfreeze the next 5 layers
        for layer in base_model.layers[i*5 : (i+1)*5]:
            layer.trainable = True

        # Create a dataset and preprocess images to suit base model
        train_ds, val_ds = create_train_val_datasets(preprocessing_model.preprocess_input, train_directory, batch_size=16)
        test_ds = create_test_dataset(preprocessing_model.preprocess_input, test_directory, batch_size=16)

        # Add a classification head
        top_model = base_model.output
        top_model = GlobalAveragePooling2D()(top_model)
        top_model = Dropout(0.5)(top_model)
        predictions = Dense(1, activation='sigmoid')(top_model)
        model = Model(inputs=base_model.input, outputs=predictions)

        # Compile the model
        model.compile(
            optimizer=Adam(1e-3),  
            loss=BinaryCrossentropy(from_logits=False),
            metrics=[BinaryAccuracy()],
        )

        # Suspending saves for now to save hard drive space. (If you uncomment be sure to update the callbacks below)
        '''
        # Setup Model Checkpoint Callback
        model_checkpoint = ModelCheckpoint(
        f'{experiment_directory}/models/{pretrained_model.__name__}/{i*5}_layers_unfrozen/best_model.h5', 
        monitor='val_loss', 
        mode='min', 
        verbose=1, 
        save_best_only=True
        )
        '''
        # Train the model
        history = model.fit(
            train_ds, 
            steps_per_epoch=train_ds.samples // train_ds.batch_size,
            epochs=4, 
            validation_data=val_ds,
            validation_steps=val_ds.samples // val_ds.batch_size,
            callbacks = [early_stopping, lr_reducer]
        )

        # Evaluate the model
        evaluate = model.evaluate(test_ds, steps = test_ds.samples // test_ds.batch_size, verbose=1)

        # Append results to accuracy_test list
        accuracy_test.append({
            'base_model': pretrained_model.__name__,
            'total_layers': len(base_model.layers),
            'layers_unfrozen': i*5,
            'processing_time': round(processing_time, 3),
            'test_accuracy': round(evaluate[1], 3),
            'test_loss': round(evaluate[0], 3),
            'train_accuracy': round(history.history['binary_accuracy'][-1],3),
            'train_loss': round(history.history['loss'][-1],3),
            'val_accuracy': round(history.history['val_binary_accuracy'][-1],3),
            'val_loss': round(history.history['val_loss'][-1],3)
        })


# Create DataFrame with accuracy score
accuracy_test_results = pd.DataFrame(accuracy_test)
accuracy_test_results.to_csv(f'{results_directory}/accuracy_test_results.csv', index=False)


Found 9660 images belonging to 2 classes.
Found 2414 images belonging to 2 classes.
Found 2139 images belonging to 2 classes.
Epoch 1/4


2023-05-24 13:55:32.680521: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


603/603 [==============================] - ETA: 0s - loss: 0.6844 - binary_accuracy: 0.5601

2023-05-24 13:56:56.602996: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss did not improve from 0.59257
603/603 [==============================] - 90s 147ms/step - loss: 0.6844 - binary_accuracy: 0.5601 - val_loss: 0.6244 - val_binary_accuracy: 0.6692 - lr: 0.0010
Epoch 2/4
603/603 [==============================] - ETA: 0s - loss: 0.6317 - binary_accuracy: 0.6396
Epoch 2: val_loss did not improve from 0.59257
603/603 [==============================] - 88s 146ms/step - loss: 0.6317 - binary_accuracy: 0.6396 - val_loss: 0.6121 - val_binary_accuracy: 0.6508 - lr: 0.0010
Epoch 3/4
603/603 [==============================] - ETA: 0s - loss: 0.5933 - binary_accuracy: 0.6750
Epoch 3: val_loss improved from 0.59257 to 0.56219, saving model to /home/luke/Development/deepfake-detection/experiments/0002/models/ResNet50/untrainable/best_model.h5
603/603 [==============================] - 85s 142ms/step - loss: 0.5933 - binary_accuracy: 0.6750 - val_loss: 0.5622 - val_binary_accuracy: 0.7033 - lr: 0.0010
Epoch 4/4
603/603 [==============================

...

2023-05-24 15:59:40.307007: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 128.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


594/603 [============================>.] - ETA: 1s - loss: 0.0249 - binary_accuracy: 0.9916

2023-05-24 16:00:54.258719: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.50MiB (rounded to 13107200)requested by op model_26/block14_sepconv2_bn/FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-24 16:00:54.258808: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-05-24 16:00:54.258817: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 288, Chunks in use: 288. 72.0KiB allocated for chunks. 72.0KiB in use in bin. 9.9KiB client-requested in use in bin.
2023-05-24 16:00:54.258823: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 67, Chunks in use: 66. 38.0KiB allocated for chunks. 37.2KiB in use in bin. 33.0KiB client-requested in use in bin.
2023-05-24 16:0

ResourceExhaustedError: Graph execution error:

Detected at node 'model_26/block14_sepconv2_bn/FusedBatchNormV3' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/luke/Development/deepfake-detection-v2/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/luke/Development/deepfake-detection-v2/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/luke/Development/deepfake-detection-v2/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/luke/Development/deepfake-detection-v2/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in st

.308803: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 79872 totalling 78.0KiB
2023-05-24 16:00:54.308808: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 98816 totalling 96.5KiB
2023-05-24 16:00:54.308813: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 11 Chunks of size 131072 totalling 1.38MiB
2023-05-24 16:00:54.308818: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 134656 totalling 131.5KiB
2023-05-24 16:00:54.308824: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 142592 totalling 139.2KiB
2023-05-24 16:00:54.308829: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 144384 totalling 141.0KiB
2023-05-24 16:00:54.308834: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 153344 totalling 149.8KiB
2023-05-24 16:00:54.308839: I tensorflow/tsl/framework/bfc_allocator.cc:1103] 1 Chunks of size 155392 totalling 151.8KiB
2023-05-24 16:00:54.308845: I tensorflow/tsl/framework/

In [11]:
accuracy_test_results = pd.DataFrame(accuracy_test)
accuracy_test_results.to_csv(f'{results_directory}/accuracy_test_results.csv', index=False)

print(accuracy_test_results)

   base_model  total_layers  layers_unfrozen  processing_time  test_accuracy   
0    Xception           132                0          317.895          0.708  \
1    Xception           132                5          317.895          0.735   
2    Xception           132               10          317.895          0.816   
3    Xception           132               15          317.895          0.808   
4    Xception           132               20          317.895          0.891   
5    Xception           132               25          317.895          0.860   
6    Xception           132               30          317.895          0.920   
7    Xception           132               35          317.895          0.875   
8    Xception           132               40          317.895          0.918   
9    Xception           132               45          317.895          0.924   
10   Xception           132               50          317.895          0.926   
11   Xception           132             